<a href="https://colab.research.google.com/github/Sundaynot/HP_Big_data_project/blob/main/HP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pyspark
!pip install -U -q PyDrive2
#l'output 'xxx is not a symbolic link' non influirà sull'implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', commenta le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla
#!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
#!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
#!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
#!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
#!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
#!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#Se si verifica un errore relativo all'esecuzione sopra indicata, puoi provare a commentare le 12 righe sopra sotto l'installazione di
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#l'output 'xxx is not a symbolic link' non influirà sulla tua implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', puoi commentare le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla

ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc_proxy.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_5.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbb.so.12': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_0.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind.so.3': File exists
openjdk-8-jdk-headless is already the newest version (8u462-ga~us1-0ubuntu2~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [26]:
# Creazione della SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("HP_Analysis") \
    .getOrCreate()

print("Sessione creata")

Sessione creata


In [27]:
# Nome della cartella e URL del repo
repo_name = "HP_Big_data_project"
repo_url = "https://github.com/sundaynot/HP_Big_data_project.git"

# Clona il repo solo se non esiste già
if not os.path.exists(repo_name):
    print(f"Clonazione del repository '{repo_name}'...")
    !git clone {repo_url}
else:
    print(f"Repository '{repo_name}' già presente.")

Repository 'HP_Big_data_project' già presente.


In [28]:
# importo librerie utili
from pyspark.sql.functions import col, lower, regexp_replace, explode, split, concat_ws, monotonically_increasing_id, lag, when

In [29]:
df_hp1 = spark.read.text("/content/HP_Big_data_project/database/01 Harry Potter and the Sorcerers Stone.txt")
df_hp2 = spark.read.text("/content/HP_Big_data_project/database/02 Harry Potter and the Chamber of Secrets.txt")
df_hp3 = spark.read.text("/content/HP_Big_data_project/database/03 Harry Potter and the Prisoner of Azkaban.txt")
df_hp4 = spark.read.text("/content/HP_Big_data_project/database/04 Harry Potter and the Goblet of Fire.txt")
df_hp5 = spark.read.text("/content/HP_Big_data_project/database/05 Harry Potter and the Order of the Phoenix.txt")
df_hp6 = spark.read.text("/content/HP_Big_data_project/database/06 Harry Potter and the Half-Blood Prince.txt")
df_hp7 = spark.read.text("/content/HP_Big_data_project/database/07 Harry Potter and the Deathly Hallows.txt")

In [30]:
df_hp1.show(1, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or

In [31]:
df_hp2.show(1, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Not for the first time, an argument had broken out over breakfast at number four, Privet Drive. Mr. Vernon Dursley had been woken in the early hours of the morning by a loud, hooting noise from his nephew Harry’s room.|
+-------------------------------------------------------------------------------------------------------------------

In [32]:
df_hp3.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his h

In [33]:
df_hp4.show(1, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------

In [34]:
df_hp5.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                            

In [35]:
df_hp6.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                    

In [36]:
df_hp7.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [78]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# se una riga è del tipo CHAPTER....  mette true nella colonna is_chapter_line
df_hp1_with_titles = df_hp1.withColumn(
    "is_chapter_line",
    F.col("value").rlike(r"^(CHAPTER\s+[A-Z0-9]+)$")
)

# Crea una finestra per mantenere l’ordine originale del testo
window_spec = Window.orderBy(F.monotonically_increasing_id())

# parte da 0 se nella colonna is_chapter_line=true aggiunge 1
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_id",
    F.sum(F.when(F.col("is_chapter_line"), 1).otherwise(0)).over(window_spec)
)

# Aggiungi anche la riga successiva come titolo del capitolo
# (ogni riga dopo "CHAPTER ..." contiene il sottotitolo, lead legge la riga successiva)
lead_title = F.lead("value").over(window_spec)

#concatena CHAPTER con il sottotitolo quando is_chapter_line=true
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_name",
    F.when(F.col("is_chapter_line"),
           F.concat_ws(" — ", F.col("value"), lead_title))
)



df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_name",
    F.when(
        (F.col("chapter_id") == 0) & F.col("chapter_name").isNull(),
        F.lit("CHAPTER ONE")
    ).otherwise(F.col("chapter_name"))
)

#Propaga il titolo del capitolo a tutte le righe successive
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_title",
    F.last("chapter_name", ignorenulls=True).over(window_spec)
)

#Raggruppa per capitolo
df_hp1_chapters = (
    df_hp1_with_titles
    .groupBy("chapter_id", "chapter_title")
    .agg(F.collect_list("value").alias("lines"))
    .withColumn("chapter_text", F.concat_ws(" ", F.col("lines")))
    .select("chapter_id", "chapter_title", "chapter_text")
    .orderBy("chapter_id")
)


+-----------------+---------------+----------+--------------------+
|value            |is_chapter_line|chapter_id|chapter_name        |
+-----------------+---------------+----------+--------------------+
|CHAPTER TWO      |true           |1         |CHAPTER TWO —       |
|CHAPTER THREE    |true           |2         |CHAPTER THREE —     |
|CHAPTER FOUR     |true           |3         |CHAPTER FOUR —      |
|CHAPTER FIVE     |true           |4         |CHAPTER FIVE —      |
|CHAPTER SIX      |true           |5         |CHAPTER SIX —       |
|CHAPTER SEVEN    |true           |6         |CHAPTER SEVEN —     |
|CHAPTER EIGHT    |true           |7         |CHAPTER EIGHT —     |
|CHAPTER NINE     |true           |8         |CHAPTER NINE —      |
|CHAPTER TEN      |true           |9         |CHAPTER TEN —       |
|CHAPTER ELEVEN   |true           |10        |CHAPTER ELEVEN —    |
|CHAPTER TWELVE   |true           |11        |CHAPTER TWELVE —    |
|CHAPTER THIRTEEN |true           |12        |CH

In [75]:
df_hp1_chapters.show()

+----------+--------------------+--------------------+
|chapter_id|       chapter_title|        chapter_text|
+----------+--------------------+--------------------+
|         0|         CHAPTER ONE|M r. and Mrs. Dur...|
|         1|      CHAPTER TWO — |CHAPTER TWO     T...|
|         2|    CHAPTER THREE — |CHAPTER THREE    ...|
|         3|     CHAPTER FOUR — |CHAPTER FOUR     ...|
|         4|     CHAPTER FIVE — |CHAPTER FIVE     ...|
|         5|      CHAPTER SIX — |CHAPTER SIX     T...|
|         6|    CHAPTER SEVEN — |CHAPTER SEVEN    ...|
|         7|    CHAPTER EIGHT — |CHAPTER EIGHT    ...|
|         8|     CHAPTER NINE — |CHAPTER NINE     ...|
|         9|      CHAPTER TEN — |CHAPTER TEN     H...|
|        10|   CHAPTER ELEVEN — |CHAPTER ELEVEN   ...|
|        11|   CHAPTER TWELVE — |CHAPTER TWELVE   ...|
|        12| CHAPTER THIRTEEN — |CHAPTER THIRTEEN ...|
|        13| CHAPTER FOURTEEN — |CHAPTER FOURTEEN ...|
|        14|  CHAPTER FIFTEEN — |CHAPTER FIFTEEN  ...|
|        1

In [80]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# Aggiungi la colonna row_id con un ID che aumenta per garantire che l'ordine di lettura originale sia preservato
df_hp1_ordered = df_hp1.withColumn("row_id", F.monotonically_increasing_id())

# FILTRA LE RIGHE VUOTE !!
# Questo è il passaggio chiave. Eliminiamo le righe che sono null, vuote ("")
# o contengono solo spazi bianchi (trim(...) == "").
df_hp1_cleaned = df_hp1_ordered.filter(
    (F.col("value").isNotNull()) & (F.trim(F.col("value")) != "")
)

# Usando F.upper(), non ci preoccupiamo più di "CHAPTER" vs "chapter".
df_hp1_with_titles = df_hp1_cleaned.withColumn(
    "is_chapter_line",
    F.upper(F.col("value")).rlike(r"^(CHAPTER\s+[A-Z0-9]+)$")
)

# Uso 'row_id' per mantenere l'ordine originale
window_spec = Window.orderBy("row_id")

# parte da 0 se nella colonna is_chapter_line=true aggiunge 1
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_id",
    F.sum(F.when(F.col("is_chapter_line"), 1).otherwise(0)).over(window_spec)
)

# F.lead() mi restituisce la riga successiva come titolo del capitolo
# (ogni riga dopo "CHAPTER ..." contiene il sottotitolo, lead legge la riga successiva)
lead_title = F.lead("value").over(window_spec)

# Creiamo un "marcatore" del nome del capitolo.
# Questo assegna il titolo (es. "THE BOY WHO LIVED") alla riga "CHAPTER ONE"
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_name_marker",
    F.when(F.col("is_chapter_line"), lead_title)
)

# Propaga il nome del capitolo (es. "THE BOY WHO LIVED") a tutte le righe
# successive che appartengono a quel capitolo.
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "chapter_title",
    F.last("chapter_name_marker", ignorenulls=True).over(window_spec)
)

# AGGREGAZIONE FINALE

# Ora dobbiamo filtrare le righe che non sono "testo"
# Vogliamo rimuovere:
# 1. Le righe "CHAPTER..." (is_chapter_line == True)
# 2. Le righe del TITOLO (la riga dopo "CHAPTER...")

# 'is_title_line' è True se la riga PRECEDENTE era "is_chapter_line"
df_hp1_with_titles = df_hp1_with_titles.withColumn(
    "is_title_line",
    F.lag("is_chapter_line", 1, False).over(window_spec)
)

# Filtra solo il testo del capitolo (escludi pre-capitolo e righe di metadati)
df_hp1_final_text = df_hp1_with_titles.filter(
    (F.col("is_chapter_line") == False) &
    (F.col("is_title_line") == False) &
    (F.col("chapter_id") > 0) # Esclude il testo prima del Capitolo 1
)

# Raggruppa per capitolo
df_hp1_chapters = (
    df_hp1_final_text
    .groupBy("chapter_id", "chapter_title")
    .agg(F.collect_list("value").alias("lines")) # Raccoglie le righe
    .withColumn("chapter_text", F.concat_ws(" ", F.col("lines"))) # Unisce in un unico testo
    .select("chapter_id", "chapter_title", "chapter_text")
    .orderBy("chapter_id")
)

# Ora questo dovrebbe funzionare
df_hp1_chapters.show(truncate=10)

+----------+-------------+------------+
|chapter_id|chapter_title|chapter_text|
+----------+-------------+------------+
|         1|   THE VAN...|  N early...|
|         2|   LETTERS...|  T he es...|
|         3|   THE KEE...|  B OOM. ...|
|         4|   DIAGON ...|  H arry ...|
|         5|   THE JOU...|  H arry’...|
|         6|   THE SOR...|  T he do...|
|         7|   THE POT...|  T here,...|
|         8|   THE MID...|  H arry ...|
|         9|    HALLOWEEN|  M alfoy...|
|        10|    QUIDDITCH|  A s the...|
|        11|   THE MIR...|  C hrist...|
|        12|   NICHOLA...|  D umble...|
|        13|   NORBERT...|  Q uirre...|
|        14|   THE FOR...|  T hings...|
|        15|   THROUGH...|  I n yea...|
|        16|   THE MAN...|  I t was...|
+----------+-------------+------------+



In [84]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from functools import reduce

#definisco una funzione da fare su tutti e 7 i libri per dividere in capitoli

def process_book_chapters(df_raw, book_number):
    """
    Processa un DataFrame grezzo di testo di un libro e lo segmenta
    in capitoli, assumendo che il testo inizi SEMPRE con il Capitolo 1
    (anche senza un marcatore esplicito "CHAPTER ONE").
    """

    # --- FASE 1: associo ad ogni riga un id per mantenere l'ordinamento ed elimino le righe vuote
    df_ordered = df_raw.withColumn("row_id", F.monotonically_increasing_id())
    df_cleaned = df_ordered.filter(
        (F.col("value").isNotNull()) & (F.trim(F.col("value")) != "")
    )

    # creo una finestra ed uso 'row_id' per mantenere l'ordine originale
    window_spec = Window.orderBy("row_id")

    # --- FASE 2: gestione del primo capitolo (che non ha nessun marker)

    # Cattura il titolo del Cap 1 (prima riga di testo pulito)
    first_line_title = df_cleaned.first()["value"]

    # --- FASE 3: MARCATORI PER CAPITOLI SUCCESSIVI ---

    # mette true nella colonna is_new_chapter_line nelle righe che sono come la regex
    chapter_regex = r"^(CHAPTER\s+[A-Z0-9]+)$"  # Questa regex cerca TUTTI i marcatori "CHAPTER...",
    df_with_markers = df_cleaned.withColumn(
        "is_new_chapter_line",
        F.upper(F.col("value")).rlike(chapter_regex)
    )

    # --- FASE 4: partendo da 1 assegno un ID crescente ad ogni capitolo

    # Assegna ID (inizia da 1).
    # Aggiunge 1 solo quando la colonna is_new_chapter_line= true
    df_with_ids = df_with_markers.withColumn(
        "chapter_id",
        F.lit(1) + F.sum(F.when(F.col("is_new_chapter_line"), 1).otherwise(0)).over(window_spec)
    )

    # Cattura i titoli dei capitoli successivi (la riga dopo il marcatore) e li inserisce nella colonna chapter_name_marker
    lead_title = F.lead("value").over(window_spec)
    df_with_name_markers = df_with_ids.withColumn(
        "chapter_name_marker",
        F.when(F.col("is_new_chapter_line"), lead_title)
    )

    # Propaga i titoli
    df_with_propagated_titles = df_with_name_markers.withColumn(
        "chapter_title_propagated",
        F.last("chapter_name_marker", ignorenulls=True).over(window_spec)
    )

    # Inserisce il titolo del Capitolo 1 manualmente
    df_with_final_titles = df_with_propagated_titles.withColumn(
        "chapter_title",
        F.when(
            F.col("chapter_id") == 1,
            F.lit(first_line_title) # Usa il titolo catturato dalla prima riga
        ).otherwise(
            F.col("chapter_title_propagated") # Usa i titoli propagati per gli altri
        )
    )

    # --- FASE 5: FILTRO E AGGREGAZIONE ---

    # Identifica le righe di metadati da rimuovere:
    # 1. Le righe "CHAPTER..."
    # 2. Le righe del TITOLO (quelle subito dopo "CHAPTER...")

    # Se è una riga titolo che segue un marcatore metti True senno False
    df_with_meta_flags = df_with_final_titles.withColumn(
        "is_title_line", # È una riga titolo che segue un marcatore?
        F.lag("is_new_chapter_line", 1, False).over(window_spec)
    )

    # Filtro finale:
    # Rimuove le righe "CHAPTER..." e le righe titolo che seguono.
    # Il testo del Capitolo 1 (inclusa la sua prima riga/titolo) viene
    # automaticamente preservato perché non ha marcatori.
    df_final_text = df_with_meta_flags.filter(
        (F.col("is_new_chapter_line") == False) & (F.col("is_title_line") == False)
    )

    # --- FASE 6: AGGREGAZIONE ---
    df_chapters = (
        df_final_text
        .groupBy("chapter_id", "chapter_title")
        .agg(F.collect_list("value").alias("lines"))
        .withColumn("chapter_text", F.concat_ws(" ", F.col("lines")))
        # Aggiungi il numero del libro per l'unione finale
        .withColumn("book_id", F.lit(book_number))
        .select("book_id", "chapter_id", "chapter_title", "chapter_text")
        .orderBy("chapter_id")
    )

    return df_chapters

In [85]:
# Metti tutti i tuoi DataFrame in una lista NELL'ORDINE CORRETTO
all_hp_dfs = [df_hp1, df_hp2, df_hp3, df_hp4, df_hp5, df_hp6, df_hp7]

processed_books_list = []

print("Inizio elaborazione dei 7 libri (con logica unificata)...")

# Usiamo enumerate per ottenere l'indice (i) e il numero del libro (i+1)
for i, df_book_raw in enumerate(all_hp_dfs):
    book_number = i + 1

    print(f"Elaborazione: Libro {book_number}...")

    # Chiama la nostra funzione unificata
    df_processed = process_book_chapters(df_book_raw, book_number)

    processed_books_list.append(df_processed)

print("Elaborazione completata.")

# --- Unisci tutto in un Unico DataFrame ---

all_chapters_df = reduce(DataFrame.unionAll, processed_books_list)

all_chapters_df.cache()

print(f"Numero totale di capitoli estratti: {all_chapters_df.count()}")

# Visualizza il risultato finale
# Ora avrai capitoli da tutti i 7 libri, identificati da "book_id"
all_chapters_df.orderBy("book_id", "chapter_id").show(truncate=80)

Inizio elaborazione dei 7 libri (con logica unificata)...
Elaborazione: Libro 1...
Elaborazione: Libro 2...
Elaborazione: Libro 3...
Elaborazione: Libro 4...
Elaborazione: Libro 5...
Elaborazione: Libro 6...
Elaborazione: Libro 7...
Elaborazione completata.
Numero totale di capitoli estratti: 97
+-------+----------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|book_id|chapter_id|                                                                   chapter_title|                                                                    chapter_text|
+-------+----------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|      1|         1|M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that t...|M r. and Mrs. Dursley, of number four, Privet Drive, 

In [90]:
from pyspark.sql import functions as F
from pyspark.ml.feature import StopWordsRemover # Importa la classe

# 1. Suddividi il testo in parole (tokenizzazione)
df_words = all_chapters_df.select(
    "book_id",
    "chapter_id",
    F.explode(F.split(F.lower(F.col("chapter_text")), r"\s+")).alias("word")
)

# 2. Pulisci le parole (normalizzazione)
df_cleaned_words = df_words.withColumn(
    "word",
    F.regexp_replace(F.col("word"), r"[^\w]", "")
).filter(F.col("word") != "").filter(F.length(F.col("word")) > 2)

# 3. RAGGRUPPA le parole in un array per capitolo
#    (Questo è necessario per StopWordsRemover)
df_word_arrays = df_cleaned_words.groupBy("book_id", "chapter_id").agg(
    F.collect_list("word").alias("words_array")
)

# 4. Inizializza e APPLICA StopWordsRemover
remover = StopWordsRemover(
    inputCol="words_array",         # Colonna da cui leggere
    outputCol="filtered_words",     # Colonna in cui scrivere
    language="english"              # Imposta la lingua inglese
)

# Aggiungi parole personalizzate da rimuovere (opzionale ma consigliato)
# "harry", "ron", "hermione" non sono stop words standard
remover.setStopWords(remover.getStopWords() + ["harry", "ron", "hermione", "dumbledore", "snape"])

# Applica la trasformazione
df_filtered_arrays = remover.transform(df_word_arrays)

# 5. RIESPLODI l'array filtrato per il conteggio finale
df_meaningful_words = df_filtered_arrays.select(
    F.explode(F.col("filtered_words")).alias("word")
)

# 6. Esegui il conteggio (Word Count)

TypeError: StopWordsRemover.__init__() got an unexpected keyword argument 'language'